In [183]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from IPython.display import display, HTML

In [145]:
class DataFrameExtended(pd.DataFrame):
    def basic_descriptives(self):
        display(HTML("<p>Head:</p>"))
        display(HTML(self.head().to_html()))
        display(HTML("<p>Summary stats:</p>"))
        display(HTML(self.describe().to_html()))
        print(
            "\nNrows: ", self.shape[0], "\n",
            "\nNcols: ", self.shape[1], "\n",
            "\nData types:\n", self.dtypes.to_string(), "\n",
            sep=''
        )

In [146]:
data = pd.read_csv('train.csv')
data = DataFrameExtended(data)

In [147]:
data.basic_descriptives()  

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200



Nrows: 891

Ncols: 12

Data types:
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object



In [241]:
# what to do with 0-1 binomial -- recode as factors
# what to do with dates?
def show_basic_plots(df, target_var=None, vars_subset=None, use_sublot=True):
    # Presets:
    if vars_subset is None:
        vars_subset = df.columns.tolist()
        
    if target_var is None:
        target_var = df.columns[0]
    
    if target_var in vars_subset:
        vars_subset.remove(target_var)
    
    color_background = '#F5F5F5'
    color_gridlines = '#DCDCDC'
    colors_in_use = ['#2C3E50', '#537EA2', '#858F84', '#42A593',
                     '#873E23', '#CFD1A1', '#6A744F', '#BDBDC5',
                     '#7EA253', '#EDB676', '#C26D40']+px.colors.qualitative.Safe
    
    dtypes_num = ['int64', 'int32', 'int16', 'float64', 'float32', 'float16']
    dtypes_str = ['object', 'category']
    vars_num = df.loc[:, vars_subset].select_dtypes(include=dtypes_num).columns
    vars_str = df.loc[:, vars_subset].select_dtypes(include=dtypes_str).columns
    
    # For categorical variables:
    fig_str_lst = [None]*len(vars_str)
    for i in range(0, len(vars_str)):
        fig = go.Figure()
        fig.add_trace(go.Histogram(x=df.loc[:, vars_str[i]],
                                   name=vars_str[i],
                                   showlegend=True))
        fig.update_traces(marker_color=colors_in_use[0],
                          marker_line_color='rgb(8,48,107)',
                          marker_line_width=1.5,
                          opacity=0.8)
        fig.update_layout(xaxis_type='category',
                          xaxis_title=vars_str[i],
                          paper_bgcolor=color_background,
                          plot_bgcolor=color_background)
        fig.update_yaxes(gridcolor=color_gridlines)
        fig.update_xaxes(linecolor=color_gridlines)
        fig_str_lst[i] = fig
        
    # For categorical variables (2+ vars):
    fig_str_lst_mix = []
    for i in range(0, len(vars_str)):
        for j in range(0, len(vars_str)):
            if i == j:
                continue
            else:
                fig = px.histogram(df, x=vars_str[i], color=vars_str[j],
                                   color_discrete_sequence=colors_in_use[1:])
                fig.update_traces(marker_line_color='rgb(8,48,107)',
                                  marker_line_width=1.5,
                                  opacity=0.8)
                fig.update_layout(xaxis_type='category',
                                  xaxis_title=vars_str[i],
                                  paper_bgcolor=color_background,
                                  plot_bgcolor=color_background)
                fig.update_yaxes(gridcolor=color_gridlines, title='')
                fig.update_xaxes(linecolor=color_gridlines)
                fig_str_lst_mix.append(fig)
            
    # For categorical versus target:
    fig_str_lst_vs_y = [None]*len(vars_str)
    if df[target_var].dtype in dtypes_str:
        for i in range(0, len(vars_str)):
            fig = px.histogram(df, x=target_var, color=vars_str[i],
                               color_discrete_sequence=colors_in_use[1:])
            fig.update_traces(marker_line_color='rgb(8,48,107)',
                              marker_line_width=1.5,
                              opacity=0.8)
            fig.update_layout(xaxis_type='category',
                              xaxis_title=target_var,
                              paper_bgcolor=color_background,
                              plot_bgcolor=color_background)
            fig.update_yaxes(gridcolor=color_gridlines, title='')
            fig.update_xaxes(linecolor=color_gridlines)
            fig_str_lst_vs_y[i] = fig
    else:
        # numerical plots split by color
        return None
    
    
    # Numerical 
    fig_num_lst_dist = [None]*len(vars_num)
    for i in range(0, len(vars_num)):
        fig = ff.create_distplot(hist_data=[df[vars_num[i]].dropna()],
                                 group_labels=[vars_num[i]],
                                 show_hist=False,
                                 show_rug=False,
                                 colors = colors_in_use)
        fig.update_layout(paper_bgcolor=color_background,
                          plot_bgcolor=color_background)
        fig.update_yaxes(gridcolor=color_gridlines,
                         zerolinecolor=color_gridlines,
                         title='')
        fig.update_xaxes(gridcolor=color_gridlines,
                         title=vars_num[i])
        fig_num_lst_dist[i] = fig
    
    # numerical box plots, violin plots;
    # numerical 2+ - scatter+lm; 2d density plots;
    # for all - density plots, box plots, violin plots, bar plots by color
    # for all 2+  -- numerical 2+ by color
    # unite at the end
    final_dict = {
        'dist_cat': fig_str_lst,
        'dist_cat_mlt': fig_str_lst_mix,
        'dist_cat_y': fig_str_lst_vs_y,
        'dist_num': fig_num_lst_dist
    }
    return(final_dict)
#fig.update_layout(barmode='stack')
#fig.update_xaxes(categoryorder='category ascending')
#fig.show()

In [242]:
tmp = show_basic_plots(data, target_var='Sex')

In [237]:
tmp['dist_num'][3]

In [231]:
color_background = '#F5F5F5'
color_gridlines = '#DCDCDC'
fig = ff.create_distplot(hist_data=[data.Age.dropna()],
                         group_labels=['Age'],
                         show_hist=False,
                         show_rug=False,
                         colors = colors_in_use)
fig.update_layout(paper_bgcolor=color_background,
                  plot_bgcolor=color_background)
fig.update_yaxes(gridcolor=color_gridlines,
                 zerolinecolor=color_gridlines,
                 title='')
fig.update_xaxes(gridcolor=color_gridlines,
                 title='Age')